# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [46]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [47]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khatanga,71.9667,102.5000,-41.45,100,77,2.12,RU,1675206759
1,1,punta de piedra,7.6176,-71.4916,27.63,44,11,0.63,VE,1675206760
2,2,bluff,-46.6000,168.3333,16.12,96,100,0.85,NZ,1675206760
3,3,alice springs,-23.7000,133.8833,23.75,88,100,2.57,AU,1675206761
4,4,ziro,27.6333,93.8333,4.12,70,10,0.96,IN,1675206761


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [48]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map plot
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[city_data_df["Max Temp"]>17,:]
ideal_df = ideal_df.loc[ideal_df["Max Temp"]<30,:]
ideal_df = ideal_df.loc[ideal_df["Humidity"]<60,:]
ideal_df = ideal_df.loc[ideal_df["Cloudiness"]<50,:]
ideal_df = ideal_df.loc[ideal_df["Wind Speed"]<5,:]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,punta de piedra,7.6176,-71.4916,27.63,44,11,0.63,VE,1675206760
114,114,dire dawa,9.5931,41.8661,19.41,48,1,1.26,ET,1675206816
117,117,tomatlan,19.9333,-105.2500,28.53,44,13,3.47,MX,1675206691
161,161,najran,17.4924,44.1277,17.12,32,19,1.23,SA,1675206839
231,231,pyinmana,19.7333,96.2167,18.53,56,19,1.12,MM,1675206878


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,punta de piedra,VE,7.6176,-71.4916,44,
114,dire dawa,ET,9.5931,41.8661,48,
117,tomatlan,MX,19.9333,-105.2500,44,
161,najran,SA,17.4924,44.1277,32,
231,pyinmana,MM,19.7333,96.2167,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "limit":limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index,"Lng"]
    latitude = hotel_df.loc[index,"Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
punta de piedra - nearest hotel: No hotel found
dire dawa - nearest hotel: Adis Abeba
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
najran - nearest hotel: No hotel found
pyinmana - nearest hotel: Myint Myat Hotel
gogrial - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
touba - nearest hotel: No hotel found
coihaique - nearest hotel: No hotel found
la palma - nearest hotel: La Quinta Inn & Suites by Wyndham Buena Park
lithgow - nearest hotel: Grand Central Hotel
kyabe - nearest hotel: No hotel found
moree - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
puerto montt - nearest hotel: Le Mirage
kutum - nearest hotel: No hotel found
calabozo - nearest hotel: Hotel Plaza Real
obo - nearest hotel: No hotel found
kapoeta - nearest hotel: Kuleu
aketi - nearest hotel: Hôtel Mabinza
koumac - nearest hotel: Monitel Koumac
san carlos de bariloche - nearest hotel: Kronos


,City,Country,Lat,Lng,Humidity,Hotel Name
1,punta de piedra,VE,7.6176,-71.4916,44,No hotel found
114,dire dawa,ET,9.5931,41.8661,48,Adis Abeba
117,tomatlan,MX,19.9333,-105.2500,44,Hotel Hacienda Vieja Tomatlan
161,najran,SA,17.4924,44.1277,32,No hotel found
231,pyinmana,MM,19.7333,96.2167,56,Myint Myat Hotel
232,gogrial,SS,8.5332,28.1004,15,No hotel found
239,kidal,ML,18.4411,1.4078,8,No hotel found
242,touba,CI,8.2833,-7.6833,16,No hotel found
270,coihaique,CL,-45.5752,-72.0662,30,No hotel found
402,la palma,US,33.8464,-118.0467,31,La Quinta Inn & Suites by Wyndham Buena Park


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.5,
    color = "Hotel Name",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)